## initialisation des classes qui se répète pour chaque acteur en suivant le même ordre

In [ ]:
#  classes=[1,2,2,10,10,3,3,11,11,4,4,5,5,6,6,13,13,7,7,12,12,8,8,9]

classes=[1,2,2,3,3,4,4,4,4,5,5,6,6,7,7,7,7,8,8,9,9,9,9,10]

In [ ]:
import torch
import numpy as np
import pandas as pd
c=[]
for i in range(0,10):
    
    for i in range(0,24):
        c.append(classes[i]-1)
ca=pd.DataFrame(c)

## imprtation des données

In [ ]:
df= pd.read_csv(r'C:\Users\amin\extracted patch\Fichier CSV\Final\Data_Final.csv')
df=df.drop(columns=['Unnamed: 0'])
df

In [ ]:
result=df
df1=result

In [ ]:
ca.reset_index(inplace=True, drop=True)
df1.reset_index(inplace=True, drop=True)
frames = [df1,ca]
result = pd.concat(frames ,axis=1,ignore_index=True)
result

## split

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
X = result.iloc[:,:-1]
y = result.iloc[:,-1]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

## normalisation

In [1]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

NameError: name 'X_train' is not defined

## modélisation

In [ ]:
import numpy as np

 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization

from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import History 
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint



In [ ]:

from keras.callbacks import History 
from keras.callbacks import LearningRateScheduler
from keras.constraints import maxnorm

epochs = 580


learning_rate = 0.01
decay_rate = decay=0.00001
momentum = 0.8

# define the optimizer function
sgd = SGD(lr=learning_rate, decay=decay_rate,momentum= 0.4, nesterov=False)

input_dim = X_train.shape[1]
num_classes = 10
batch_size = 170






Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
# 7. Define model architecture
model = Sequential()
model.add(Dense(240,  input_dim= input_dim, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(200,  input_dim= input_dim, activation = "relu"))
model.add(Dense(150,  input_dim= input_dim, activation = "relu"))
model.add(Dense(150,  input_dim= input_dim, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(120, activation = "relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(60, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(30, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(20, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))



# compile the model
model.compile(loss='categorical_crossentropy', 
                                optimizer=sgd, 

metrics=['acc'])

# define the learning rate change 
def exp_decay(epoch):
    lrate = learning_rate * np.exp(-decay_rate*epoch)
    return lrate

filepath = "model_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("model/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))   
# learning schedule callback
loss_history = History()
lr_rate = LearningRateScheduler(exp_decay)
callbacks_list = [loss_history, lr_rate]

# you invoke the LearningRateScheduler during the .fit() phase
exponential_decay_model_history = model.fit(X_train, Y_train,
                                    batch_size=batch_size,
                                    epochs=epochs,
                                    callbacks=callbacks_list,
                                    verbose=1,
                                    validation_data=(X_test, Y_test))




## evaluation du model

In [ ]:
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)
score[1]*100

In [ ]:
from matplotlib import pyplot
# evaluate the model
_, train_acc = model.evaluate(X_train, Y_train, verbose=0)
_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot history
pyplot.plot(exponential_decay_model_history.history['acc'], label='train')
pyplot.plot(exponential_decay_model_history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()